# 第3回 情報検索の評価

この演習ページでは，既存のツールを使って各種評価指標を計算する方法について説明します．

参考文献
- [情報アクセス評価方法論 -検索エンジンの進歩のために-](https://www.amazon.co.jp/%E6%83%85%E5%A0%B1%E3%82%A2%E3%82%AF%E3%82%BB%E3%82%B9%E8%A9%95%E4%BE%A1%E6%96%B9%E6%B3%95%E8%AB%96-%E9%85%92%E4%BA%95-%E5%93%B2%E4%B9%9F/dp/4339024961), 酒井哲也, コロナ社, 2015.

## ライブラリ
この演習では，情報検索におけるさまざまな評価指標を計算するためのツールキットである NTCIREVAL のPython版である pyNTCIREVAL を使用します．

- [pyNTCIREVAL](https://github.com/mpkato/pyNTCIREVAL/) by 京都大学 加藤 誠 先生
- [NTCIREVAL](http://research.nii.ac.jp/ntcir/tools/ntcireval-ja.html) by 早稲田大学 酒井 哲也 先生

NTCIREVALの説明を上記ページから引用します．

----
```
NTCIREVALは、様々な検索評価指標を計算するためのツールキットです。
NTCIRやTRECのad hoc文書検索タスクの他、diversified search resultsの評価やNTCIR-8コミュニティQAタスクの評価などにも利用できます。

NTCIREVALは例えば以下のような指標を算出できます:
-Average Precision
-Q-measure
-nDCG
-Expected Reciprocal Rank (ERR)
-Graded Average Precision (GAP)
-Rank-Biased Precision (RBP)
-Normalised Cumulative Utility (NCU)
-上記各指標の短縮リスト版
-Bpref
-D-measures and D#-measures (多様性評価用)
-Intent-Aware (IA) metrics (多様性評価用)
```
----

## ライブラリのインストール
`pip`というPythonライブラリ管理ツールを使用してインストールします． ターミナル上で h29iroのフォルダに移動し，下記コマンドで pyNTCIREVAL をインストールしてください．

```
pip install git+https://github.com/mpkato/pyNTCIREVAL.git
```

正しくインストールできれば， notebook上で

!pyNTCIREVAL

と実行すれば，以下の様なメッセージが出力されます．

```
Usage: pyNTCIREVAL [OPTIONS] COMMAND [ARGS]...

Options:
  -h, --help  Show this message and exit.

Commands:
  compute
  label
```


In [1]:
!pyNTCIREVAL

Usage: pyNTCIREVAL [OPTIONS] COMMAND [ARGS]...

Options:
  -h, --help  Show this message and exit.

Commands:
  compute
  label


なお，notebook上で $!$ の後の文字列はシェル（ターミナル）に対するコマンドと解釈され，シェルの出力がnotebookの画面に出力されます．

# 1. 評価データの準備

NTCIREVALおよびpyNTCIREVALでは，評価用のテキストファイルをプログラムに渡すことで，評価値を計算します．
`../data/eval/` にサンプルデータを置いています． 

基本的に，ある手法のある検索課題に対する評価指標を検索するためには，以下の２つのファイルを準備する必要があります．

- 適合性評価ファイル（*.rel）
- 検索結果ファイル（*.res）

## 適合性評価ファイル

適合性評価ファイルは，ある検索課題に対するコレクション中の適合性評価結果を表すテキストファイルです． `../data/eval/q1.rel` にサンプルデータを置いています．このファイル名は，検索課題$q_1$に対する適合性評価ファイルであることを意味しています（NTICREVALではファイル名に形式はありません．単純に利用者が分かりやすいため，山本がこのような名前をつけています）．

q1.rel の中身はこのようになっています．

In [2]:
!cat ../data/eval/q1.rel

d1 L1
d2 L0
d3 L2
d4 L1
d5 L0
d6 L0

このファイルの一行の意味は，

```
文書ID 適合性ラベル
```
となっています． 文書IDは評価データ作成者が適宜付与したIDです．適合性ラベルは慣習上このような書き方をしており，
L0 が不適合を表し，L1,  L2 ... , 以降は適合性の度合い（適合度）を表します．今回は，適合度は3段階（$\{0,1,2 \}$）のため，ラベルは$\{L0,L1,L2\}$の3種類です．4段階の適合度を用いる場合は，$\{L0,L1,L2,L3\}$をラベルとして用います．

たとえば， q1.relファイルの3行目の

```
d3 L2
```

は，文書ID $d_3$の適合度が $2$ であることを表しています．

## 検索結果ファイル

検索結果ファイルは，ある手法のある検索課題に対する検索結果（つまり，ランク付けされた文書集合）を表すテキストファイルです．  `../data/eval/method1.q1.res`にサンプルデータを置いています．

`method1.q1.res` の中身はこのようになっています．

In [3]:
!cat ../data/eval/method1.q1.res

d1
d2
d3

このように，検索結果ファイルはランキング結果を単純に文書IDで表します．たとえば，このファイルは， 検索課題$q_1$に対して $d_1, d_2, d_3$ の順で文書をランキングしたことを表しています．

# 2. 適合性ラベル付き検索結果ファイルの作成

適合性評価ファイルと検索結果ファイルを準備したら，次に適合性ラベル付き検索結果ファイルを作成します．このファイルはpyNTCIREVALの機能を利用して作成することができます．また，自身でプログラムを書いてこのファイルを作成することもできます． pyNTCIREVAL を用いて適合性ラベル付き検索結果ファイルを作成する場合は，以下のコマンドを実行します．

In [4]:
!pyNTCIREVAL label -r ../data/eval/q1.rel < ../data/eval/method1.q1.res

d1 L1
d2 L0
d3 L2


シェルのパイプを用いているので，シェルについて詳しくない人は上記コマンドの意味がよく分からないかもしれませんが

```
pyNTCIREVAL label
```

は適合性ラベル付き検索結果ファイルを作成するためのコマンドです．

```
-r ../data/eval/q1.rel 
```

は適合性評価ファイルの場所を指定しており，

```
 < ../data/eval/method1.q1.res
```

はpyNTCIREVALに，ラベルを付与して欲しい検索結果を渡しています．

上記コマンドを実行すると，

```
d1 L1
d2 L0
d3 L2
```

という結果が得られます．つまりこのプログラムは，検索結果ファイル中の各文書IDに対して，適合性評価ファイル中の対応する文書IDの適合性ラベルを付加しています．

ちなみに，上記コマンドだけでは適合性ラベル付き検索結果ファイルの内容が画面に出力されるだけです．この内容をファイルに保存するには，例えば以下の様にします．

In [5]:
!pyNTCIREVAL label -r ../data/eval/q1.rel < ../data/eval/method1.q1.res > ../data/eval/method1.q1.rel

```
 >  ../data/eval/method1.q1.rel
```
 
これもシェルのコマンドで，出力をmethod1.q1.relに書き込むという意味です．

In [6]:
!cat ../data/eval/method1.q1.rel

d1 L1
d2 L0
d3 L2


# 3. 評価指標の計算

適合性評価ファイルと，適合性評価付き検索結果ファイルが準備できたら，それらをpyNTCIREVALに渡すことで，各種評価指標を計算できます．

In [7]:
!pyNTCIREVAL compute -r ../data/eval/q1.rel -g 1:3 --cutoffs=1,3　< ../data/eval/method1.q1.rel

 # syslen=3 jrel=3 jnonrel=3
 # r1=1 rp=3
 RR=                 1.0000
 OMeasure=           0.5000
 PMeasure=           0.7500
 PPlusMeasure=       0.6250
 AP=                 0.5556
 QMeasure=           0.4167
 NCUguP=             0.6000
 NCUguBR=            0.5500
 NCUrbP=             0.5726
 NCUrbBR=            0.4251
 RBP=                0.0618
 ERR=                0.4375
 AP@0001=            1.0000
 QMeasure@0001=      0.5000
 nDCG@0001=          0.3333
 MSnDCG@0001=        0.3333
 Precision@0001=     1.0000
 nERR@0001=          0.3333
 Hit@0001=           1.0000
 AP@0003=            0.5556
 QMeasure@0003=      0.4167
 nDCG@0003=          0.6247
 MSnDCG@0003=        0.6052
 Precision@0003=     0.6667
 nERR@0003=          0.5490
 Hit@0003=           1.0000


## コマンドの説明

```
pyNTECIREVAL compute
```

は評価指標を計算するためのコマンドです．

```
-g 1:3
```

は適合度$L1，L2$の文書に対する利得（Gain）を指定しています．この値はたとえばnDCGを計算する際に使用されます．今回は利得関数として， $g(i) = 2^{{\rm rel}_i} -1$を用いるので， $L1 = 1, L2 = 3$という利得を指定しています．


```
--cutoffs 1:3
```

は，指標を計算する際に検索結果上位何件まで考慮するか（@$k$）を指定しています．この場合，上位1,3件における指標がそれぞれ出力されます．



## 結果の見方

各指標の結果が出力されています．たとえば，nERR@0003は上位$3$件まで考慮した際のnERRの値を表しています（すなわち，nERR@$3$）．また，@$k$が付与されていない指標は与えられたランキング結果全てを考慮した際の評価値です． 


<span style="color:red">注意点として，本講義で示したnDCGの定義は MSnDCG版と呼ばれるnDCGの計算方法です．従って，本講義のnDCG@$3$に対応する指標はMSnDCG@0003になります．</span>

最後に，評価結果をファイルに保存するには，先ほどと同様に出力をファイルに書き込みます．

In [8]:
!pyNTCIREVAL compute -r ../data/eval/q1.rel -g 1:3 --cutoffs=1,3 < ../data/eval/method1.q1.rel > ../data/eval/method1.q1.eval

pyNTCIREVALの使い方は以上となります． pyNTCIREVALの本体であるNTCIREVALには，複数の検索課題に対する複数の手法の結果を一括で処理するようなプログラムも含まれています．興味がある人，あるいは実際に研究でそのような評価実験を行う可能性がある人は，ぜひNTCIREVALについても調べてください．

----
# 演習課題その2 情報検索の評価

## 必須課題 （１） 動作の確認

このページで用いた検索結果に対するpyNTCIREVALの出力のうち，`MSnDCG@0003`と`nERR@0003`が，講義資料の定義に従った計算と一致していることを確かめよ．つまり，nDCG@3とnERR@3を計算するプログラム書き，その結果がpyNTCIREVALの結果と一致していることを確認せよ．


## 必須課題（２）独自データに対する評価指標の計算

演習課題1で扱った検索課題集合と検索結果に対して各自で評価用データを作成しpyNTCIREVALを用いて評価指標を計算せよ．そして， MRR，nDCG@3およびnERR@3の平均を報告し，それらの値の違いが各指標のどういった要因によるものか考察せよ．なお，演習課題1で扱ったコーパス以外で評価データを作成してもい．ただし，評価データはダミーデータでなく実際の何らかのランキングを評価したものとし，検索課題（クエリ）は3つ以上とする．
 
## 任意課題 （a）評価指標の調査

情報検索の評価指標にはさまざまなものがあり，今回の講義で扱わなかった指標も多い．たとえば， MAP（Mean Average Precision），RBP（Rank-Biased Precision）， Q-measureなどがある．そこで，これらの指標について2つ以上調べ，どのような指標か説明するとともに，それを計算するプログラムを作成し，pyNTCIREVALの結果と一致することを確認せよ．

- Mean Averge Precision
 - 参考: Introduction to Informaton Retrieval, Chapter 8: Evaluation in information retrieval.
- Q-measure
 - 参考: Sakai, T. and Song, R.: Evaluating Diversified Search Results Using Per-intent Graded Relevance,
SIGIR 2011.
- Rank-Biased Precision:
  - 参考: Moffat, A. and Zobel, J.: Rank-biased Precision for Measurement of Retrieval Effectiveness, ACM TOIS 27(1), 2008.


# 課題の提出方法

いずれかの方法で，ipython notebookのページ（.ipynbファイル）とそのhtml版を提出すること．

1. 添付ファイルで山本に送信．
   - 送付先 tyamamot at dl.kuis.kyoto-u.ac.jp
2. 各自のgithubやgithub gistにアップロードし，そのURLを山本に送信．この場合はhtml版を用意する必要はない．
3. 上記以外で，山本が実際に.ipynbファイルを確認できる方法．


# 締切

- 2017年11月30日（木）23:59
- 締切に関する個別の相談は``受け付けます``．
